In [2]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Note: you ma

In [3]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

from tqdm.notebook import tqdm_notebook as tqdm

import os

In [4]:
angry= '../input/dog-emotions-prediction/images/angry/'
sad = '../input/dog-emotions-prediction/images/sad/'
relaxed = '../input/dog-emotions-prediction/images/relaxed/'
happy = '../input/dog-emotions-prediction/images/happy/'


angry_path = os.listdir(angry)
sad_path = os.listdir(sad)
relaxed_path = os.listdir(relaxed)
happy_path = os.listdir(happy)

In [5]:
def load_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image,(224, 224))
    return image[...,::-1]

In [6]:
dataset_path = "/kaggle/input/dog-emotions-prediction/images"

In [7]:
data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.3)

In [8]:
train = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="training")

Found 11147 images belonging to 4 classes.


In [9]:
val = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="validation"
                                          )

Found 4774 images belonging to 4 classes.


In [10]:
mnet = MobileNetV2(include_top = False, weights = "imagenet" ,input_shape=(96,96,3))

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
tf.keras.backend.clear_session()

model = Sequential([mnet,
                    GlobalAveragePooling2D(),
                    Dense(512, activation = "relu"),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(128, activation = "relu"),
                    Dropout(0.1),
                    # Dense(32, activation = "relu"),
                    # Dropout(0.3),
                    Dense(4, activation = "sigmoid")])

model.layers[0].trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

model.summary()
Model: "sequential"

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [12]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
hist = model.fit_generator(train,
                    epochs=40,
                    callbacks=[lr_callbacks],
                    validation_data=val)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/40
349/349 [==============================] - 91s 237ms/step - loss: 1.4021 - accuracy: 0.4305 - val_loss: 1.2540 - val_accuracy: 0.4715 - lr: 0.0010
Epoch 2/40
349/349 [==============================] - 52s 149ms/step - loss: 1.1376 - accuracy: 0.5102 - val_loss: 1.2118 - val_accuracy: 0.4757 - lr: 0.0010
Epoch 3/40
349/349 [==============================] - 52s 148ms/step - loss: 1.0638 - accuracy: 0.5477 - val_loss: 1.2046 - val_accuracy: 0.4897 - lr: 0.0010
Epoch 4/40
349/349 [==============================] - 50s 144ms/step - loss: 0.9399 - accuracy: 0.6122 - val_loss: 1.1934 - val_accuracy: 0.4914 - lr: 1.0000e-04
Epoch 5/40
349/349 [==============================] - 51s 146ms/step - loss: 0.9085 - accuracy: 0.6266 - val_loss: 1.1935 - val_accuracy: 0.4981 - lr: 1.0000e-04
Epoch 6/40
349/349 [==============================] - 50s 142ms/step - loss: 0.8726 - accuracy: 0.6518 - val_loss: 1.2086 - val_accuracy: 0.5021 - lr: 1.0000e-04
Epoch 7/40
349/349 [====================

In [ ]:
img_path = '/kaggle/input/dog-emotions-prediction/images/relaxed/10011399856_62324e2bd2_b.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (96, 96))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB
img = img / 255.0 # Normalize the image

# Reshape the image to have a batch size of 1
img = np.reshape(img, (1, 96, 96, 3))

# Make the prediction
predictions = model.predict(img)

# Print the predicted class
class_names = ['angry', 'happy', 'relaxed', 'sad']
predicted_class = np.argmax(predictions)
print("Predicted class:", class_names[predicted_class])